<a href="https://colab.research.google.com/github/fowardelcac/Mercado-inmobilirario-cba/blob/main/Dataset/Remax/1_remax.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [15]:
page_number = 0
url_base = 'https://www.remax.com.ar/listings/buy?page='
url_base1 = '&pageSize=24&sort=-createdAt&in:operationId=1&in:typeId=9,10,11,1,2,3,4,5,6,7,8&locations=in::::4202@%3Cb%3ECordoba%3C%2Fb%3E#%20%3Cb%3ECapital%3C/b%3E#%20%3Cb%3EC%C3%B3rdoba%3C/b%3E:::&filterCount=1&viewMode=list'

data_list = []
while True:
    # Paso 2: Obtén el contenido de la página
    url = url_base + str(page_number) + url_base1

    result = requests.get(url)
    soup = BeautifulSoup(result.content, 'html.parser')
    # Extrae los datos relevantes de la página utilizando BeautifulSoup
    titulos = soup.find_all('h2', class_='description ng-star-inserted')
    span_valor = soup.find_all('span', class_='bold-span')

    data = {}
    data['Descripciones'] = [i.get_text() for i in titulos]
    data['M2'] = [span_valor[i].text for i in range(0, len(span_valor), 4)]
    data['M2 Cubiertos'] = [span_valor[i].text for i in range(1, len(span_valor), 4)]
    data['Ambientes'] = [span_valor[i].text for i in range(2, len(span_valor), 4)]
    data['Baños'] = [span_valor[i].text for i in range(3, len(span_valor), 4)]
    lengths = [len(data[key]) for key in data]

    if len(set(lengths)) == 1:  # Si todos los arrays tienen la misma longitud
        data_list.append(data)# Agrega NaNs para igualar la longitud
    page_number += 1
    # Paso 6: Maneja la paginación (por ejemplo, detecta la última página o establece un límite)
    if page_number == 83:  # Condición para detener la paginación
        break

In [16]:
df = pd.concat([pd.DataFrame(data) for data in data_list], ignore_index=True)

In [18]:
df.to_csv('Remax_df.csv')